# Enunciado

Gerar um texto, em português, como se fosse um atendente virtual de uma imobiliária.

Através da sintetização do texto, a AV precisa dizer ao cliente que está “ligando” o que ele pode pedir. 

São quatro áreas obrigatórias:
- Vendas
- Aluguel
- Administrativo
- Financeiro

Com o microfone, a pessoa responde uma dessas áreas e através de alguma das funções de reconhecimento de áudio, a área deverá ser identificada e, novamente por sintetização de voz, deverá gerar um áudio indicando que a ligação será transferida para a área correspondente. Por exemplo: “Ótimo, vou transferir você para um de nossos corretores de vendas.”

Deve estar em um loop. E deve haver uma palavra chave pra encerrar o loop. 

Entregar:
- Texto descrevendo o problema e a solução desenvolvida
- Código em Python
- Link para vídeo demonstrando a execução do programa

## Texto descrevendo o problema e a solução desenvolvida


Resposta:

Primeiro definimos as frases que o atendente virtual precisa falar para o atendimento. 

Depois definimos algumas funções auxiliares e por ultimo montamos o fluxo.

Enquanto o usuário não falar a palavra "Tchau" ou "Obrigado(a)" o fluxo ficara em loop esperando por uma das palavras:

- Vendas
- Aluguel
- Administrativo
- Financeiro

Se nada for dito ou a palavra dita não for reconhecida, o fluxo ira responder com uma mensagem de opção inválida.

Utilizamos a bibliotca do Google para fazer a voz sintetizada do atendante. Sobre o usuário, utilizamos a biblioteca PyAudio para gravar o som em conjunto com a biblioteca do Google para transcrever o audio.


## Código em Python

In [1]:
from IPython.display import Audio, display
import numpy as np
import speech_recognition as sr
from gtts import lang, gTTS
from pydub import AudioSegment
from scipy.io.wavfile import write
import pyaudio
import wave
import io
import ffmpeg
from time import sleep
import time
import os
from preferredsoundplayer import soundplay, stopsound, getIsPlaying

C:\Users\vinix\AppData\Roaming\Python\Python39\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


#### Mensagens

In [2]:
HELLO = "Olá, sou o robô falante da imobiliária Casas. Com qual área gostaria de falar? Administrativo, Aluguel, Financeiro ou Vendas"
HELLO2 = "Olá novamente. Com qual área gostaria de falar? Administrativo, Aluguel, Financeiro, Vendas ou diga 'sair' para encerrar a conversa"
NOT_UNDERSTAND = "Opção inválida. Por favor, escolha uma das opções: Administrativo, Aluguel, Financeiro, Vendas ou diga 'sair' para encerrar a conversa"
SALES = "Ótimo, vou transferir você para um de nossos corretores de vendas"
RENT = "Ótimo, vou transferir você para um de nossos corretores de aluguel"
ADMIN = "Ótimo, vou transferir você para o nosso departamento Administrativo"
FINANCE = "Ótimo, vou transferir você para o nosso departamento Financeiro"
THANKS = "A imobiliária Casas agradece seu contato. Obrigado por utilizar o robô falante. Até mais!"

#### Funções auxiliares

In [5]:
def get_channels():
  p = pyaudio.PyAudio()
  for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    print((i,dev['name'],dev['maxInputChannels']))

get_channels()

(0, 'Microsoft Sound Mapper - Input', 2)
(1, 'Headset Microphone (HyperX 7.1 ', 2)
(2, 'Microphone (HD Pro Webcam C920)', 2)
(3, 'Microsoft Sound Mapper - Output', 0)
(4, 'Headset Earphone (HyperX 7.1 Au', 0)
(5, 'LC27G7xT (NVIDIA High Definitio', 0)
(6, 'VS248 (NVIDIA High Definition A', 0)
(7, 'Realtek Digital Output (Realtek', 0)
(8, 'Primary Sound Capture Driver', 2)
(9, 'Headset Microphone (HyperX 7.1 Audio)', 2)
(10, 'Microphone (HD Pro Webcam C920)', 2)
(11, 'Primary Sound Driver', 0)
(12, 'Headset Earphone (HyperX 7.1 Audio)', 0)
(13, 'LC27G7xT (NVIDIA High Definition Audio)', 0)
(14, 'VS248 (NVIDIA High Definition Audio)', 0)
(15, 'Realtek Digital Output (Realtek(R) Audio)', 0)
(16, 'LC27G7xT (NVIDIA High Definition Audio)', 0)
(17, 'Headset Earphone (HyperX 7.1 Audio)', 0)
(18, 'VS248 (NVIDIA High Definition Audio)', 0)
(19, 'Realtek Digital Output (Realtek(R) Audio)', 0)
(20, 'Headset Microphone (HyperX 7.1 Audio)', 2)
(21, 'Microphone (HD Pro Webcam C920)', 2)
(22, 'Line In

In [3]:
def start_recording(seconds = 5, filename = "input.wav"):
  if os.path.exists(filename):
    os.remove(filename) 
  chunk = 1024
  sample_format = pyaudio.paInt16
  fs = 44100
  # Pode ser necessário trocar o valor do channels dependendo do dispositivo, para isso, execute a função get_channels() e veja qual número usar
  channels = 2
  p = pyaudio.PyAudio()

  print('Recording, speak now...')

  stream = p.open(
    format=sample_format,
    channels=channels,
    rate=fs,
    frames_per_buffer=chunk,
    input=True
  )

  frames = []
  for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

  stream.stop_stream()
  stream.close()
  p.terminate()

  print('Finished recording')

  wf = wave.open(filename, 'wb')
  wf.setnchannels(channels)
  wf.setsampwidth(p.get_sample_size(sample_format))
  wf.setframerate(fs)
  wf.writeframes(b''.join(frames))
  wf.close()

def speak(text, lang = 'pt', tld = 'com.br', filename = 'output.mp3'):
  if os.path.exists(filename):
    os.remove(filename) 
  tts = gTTS(text=text, lang=lang, tld=tld)
  tts.save(filename)
  sound = soundplay(filename)
  while getIsPlaying(sound):
    sleep(1)
  stopsound(sound)

def stt_google(filename, lang = 'pt-BR'):
  recog = sr.Recognizer()
  try:
    with sr.AudioFile(filename) as source:
      audio = recog.record(source)
      return recog.recognize_google(audio,language=lang)
  except:
    return ''

def exit_loop(transcript):
  if 'obrigad' in transcript.lower() or 'tchau' in transcript.lower() or 'adeus' in transcript.lower() or 'encerrar' in transcript.lower() or 'sair' in transcript.lower():
    return True
  else:
    return False

#### Lógica de funcionamento

In [13]:
speak(HELLO)
transcript = ''
while (not exit_loop(transcript)):
  start_recording()
  transcript = stt_google('input.wav')
  if 'vend' in transcript.lower():
    speak(SALES)
    sleep(1)
    speak(HELLO2)
  elif 'administrativ' in transcript.lower():
    speak(ADMIN)
    sleep(1)
    speak(HELLO2)
  elif 'aluguel' in transcript.lower():
    speak(RENT)
    sleep(1)
    speak(HELLO2)
  elif 'financeir' in transcript.lower():
    speak(FINANCE)
    sleep(1)
    speak(HELLO2)
  elif not exit_loop(transcript):
    speak(NOT_UNDERSTAND)

speak(THANKS)

Recording, speak now...
Finished recording
result2:
{   'alternative': [{'confidence': 0.82141513, 'transcript': 'financeiro'}],
    'final': True}
Recording, speak now...
Finished recording
result2:
[]
Recording, speak now...
Finished recording
result2:
{   'alternative': [{'confidence': 0.82141513, 'transcript': 'administrativo'}],
    'final': True}
Recording, speak now...
Finished recording
result2:
{   'alternative': [{'confidence': 0.82141513, 'transcript': 'sair'}],
    'final': True}


## Link para vídeo demonstrando a execução do programa

Link: https://youtu.be/WoYroUT-dP4 

OBS: por favor aumente o volume porque o volume da minha voz ficou um pouco baixo